In [1]:
import sys
sys.path.append('/home/ggsolver/')

In [2]:
import itertools
import networkx as nx
import spot

from pathlib import Path
from lark import Lark, Transformer, Tree, Visitor
from ggsolver.logic.formula import BaseFormula, PARSERS_DIR
from ggsolver.logic import ScLTLFormula
from ggsolver.logic.automata import PrefDfa, cross_product

In [4]:
ebnf_scltl = \
"""
start: wrapped_formula | formula

wrapped_formula: "(" start ")"
formula: binary_op | unary_op | atom

binary_op: and_op | or_op | until_op
unary_op: not_op | next_op | eventually_op

and_op: start "&" start
or_op: start "|" start
until_op: start "U" start

not_op: "!" atom
next_op: "X" start
eventually_op: "F" start

atom: VALID_NAME

VALID_NAME: /[a-z][a-z0-9_]*/

%ignore /\s+/
"""

In [5]:
scltl_parser = Lark(ebnf_scltl, parser="lalr")

In [6]:
scltl_parser.parse("a & b")

Tree('start', [Tree('formula', [Tree('binary_op', [Tree('and_op', [Tree('start', [Tree('formula', [Tree('atom', [Token('VALID_NAME', 'a')])])]), Tree('start', [Tree('formula', [Tree('atom', [Token('VALID_NAME', 'b')])])])])])])])

In [8]:
ebnf_pref = \
"""
start: pref_or

?pref_or: pref_and ("|" pref_and)*
?pref_and: atomic_pref ("&" atomic_pref)*
?atomic_pref: strict
            | weak
            | indifference
            | wrapped_pref
            
strict: scltl_formula ">" scltl_formula
weak: scltl_formula ">=" scltl_formula
indifference: scltl_formula "~" scltl_formula
wrapped_pref: "(" atomic_pref ")"

?scltl_formula: scltl_or
?scltl_or: scltl_and ("|" scltl_and)*
?scltl_and: scltl_until ("&" scltl_until)*
?scltl_until: scltl_unary_op ("U" scltl_unary_op)*

?scltl_unary_op: eventually
                | next
                | not
                | wrapped_scltl

eventually: "F" scltl_formula
next: "X" scltl_formula
not: "!" atom
?wrapped_scltl: atom
                | "(" scltl_formula ")"

atom: true | false | VALID_NAME

true: /(?i:true)/
false: /(?i:false)/
VALID_NAME.2: /[a-z][a-z0-9_]*/

%ignore /\s+/
"""

In [9]:
parser = Lark(ebnf_pref, parser="lalr")

In [11]:
tree = parser.parse("(F(b & c) > a) & (a >= b)")
tree

Tree('start', [Tree('pref_and', [Tree('wrapped_pref', [Tree('strict', [Tree('eventually', [Tree('scltl_and', [Tree('atom', [Token('VALID_NAME', 'b')]), Tree('atom', [Token('VALID_NAME', 'c')])])]), Tree('atom', [Token('VALID_NAME', 'a')])])]), Tree('wrapped_pref', [Tree('weak', [Tree('atom', [Token('VALID_NAME', 'a')]), Tree('atom', [Token('VALID_NAME', 'b')])])])])])